<a href="https://colab.research.google.com/github/fatmamoussa/chomage-en-grand-tunis-R/blob/main/ProjetR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages(c("tidyverse","survey","janitor","skimr"))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’, ‘snakecase’




In [ ]:
# installer / charger les paquets
if (!require(googledrive)) install.packages("googledrive")
if (!require(readr)) install.packages("readr")

library(googledrive)
library(readr)

# si le fichier est partagé publiquement (ou accessible avec ton compte) :
drive_deauth()   # si le fichier est public; sinon, omettre ou authentifier
file_id <- "https://drive.google.com/file/d/1FkjjDM4ZNBNGBbPBkoQ-e5B09ErKcAOp/view?usp=sharing"  # l’ID de ton lien
public_file <- drive_get(as_id(file_id))

# Télécharger le fichier localement (optionnel mais simple)
drive_download(public_file, path = "Data_T4_2023.csv", overwrite = TRUE)

# Lire en tant que dataframe
df <- read_csv("Data_T4_2023.csv", show_col_types = FALSE)



Loading required package: googledrive

Loading required package: readr

File downloaded:

• Data_T4_2023.csv <id: 1FkjjDM4ZNBNGBbPBkoQ-e5B09ErKcAOp>

Saved locally as:

• Data_T4_2023.csv



In [ ]:

summary(df)



   Y_Quarter          Year         Quarter     Region              HH_Id     
 Min.   :20234   Min.   :2023   Min.   :4   Length:67444       Min.   :   1  
 1st Qu.:20234   1st Qu.:2023   1st Qu.:4   Class :character   1st Qu.: 717  
 Median :20234   Median :2023   Median :4   Mode  :character   Median :1413  
 Mean   :20234   Mean   :2023   Mean   :4                      Mean   :1444  
 3rd Qu.:20234   3rd Qu.:2023   3rd Qu.:4                      3rd Qu.:2089  
 Max.   :20234   Max.   :2023   Max.   :4                      Max.   :3678  
                                                                             
 Member_Number    Relationship          Gender               Age        
 Min.   : 1.000   Length:67444       Length:67444       Min.   :  0.00  
 1st Qu.: 1.000   Class :character   Class :character   1st Qu.: 15.00  
 Median : 2.000   Mode  :character   Mode  :character   Median : 35.00  
 Mean   : 2.599                                         Mean   : 35.78  
 3rd Qu.: 4

In [ ]:
# Exclure les âges impossibles
df <- df %>% filter(Age >= 15 & Age <= 100)

# Exclure les Status_Work incohérents
df <- df %>% filter(Status_Work %in% c(1,2,3,4,5,6))


ERROR: Error: object 'Age' not found


In [ ]:
df$Edu_Year[is.na(df$Edu_Year)] <- 0 # 0 = inconnu


In [ ]:
df$Gender <- toupper(trimws(df$Gender))


Taux de chômage selon Région

In [ ]:
library(dplyr)

# Filtrer population active (employed + unemployed)
active <- df %>% filter(Status_Work %in% c(1,2))

# Taux de chômage par région
chomage_region <- active %>%
  group_by(Region) %>%
  summarise(
    Population_Active = n(),
    Chomeurs = sum(Status_Work == 2, na.rm=TRUE),
    Taux_Chomage = round(100 * Chomeurs / Population_Active, 2)
  )

chomage_region


Taux d’activité par région (hommes/femmes)


Taux d’activité hommes / femmes

In [ ]:
# Population 15 ans et plus
pop_15plus <- df %>% filter(Age >= 15)

taux_activite <- pop_15plus %>%
  group_by(Gender) %>%
  summarise(
    Population_15plus = n(),
    Population_Active = sum(Status_Work %in% c(1,2), na.rm=TRUE),
    Taux_Activite = round(100 * Population_Active / Population_15plus,2)
  )

taux_activite


In [ ]:
library(dplyr)
library(ggplot2)

# On travaille sur population 15+
pop_15plus <- df %>% filter(Age >= 15)

# Calcul du taux d'activité par ménage et par genre
taux_activite_hh <- pop_15plus %>%
  group_by(HH_Id, Gender) %>%
  summarise(
    Population_15plus = n(),
    Population_Active = sum(Status_Work %in% c(1,2), na.rm=TRUE),
    Taux_Activite = Population_Active / Population_15plus
  ) %>%
  ungroup()


In [ ]:
ggplot(taux_activite_hh, aes(x = Gender, y = Taux_Activite, fill = Gender)) +
  geom_boxplot() +
  scale_y_continuous(labels = scales::percent) +
  labs(
    title = "Taux d'activité par genre (par ménage)",
    x = "Genre",
    y = "Taux d'activité"
  ) +
  theme_minimal() +
  theme(legend.position = "none")


Chômage selon niveau d’éducation

In [ ]:
chomage_education <- df %>%
  filter(Status_Work %in% c(1,2)) %>%
  group_by(Educa_Level) %>%
  summarise(
    Population_Active = n(),
    Chomeurs = sum(Status_Work == 2, na.rm=TRUE),
    Taux_Chomage = round(100 * Chomeurs / Population_Active,2)
  )

chomage_education


In [ ]:
# Assurer que la colonne Chomeur existe
pop_15plus <- pop_15plus %>%
  mutate(Chomeur = ifelse(Status_Work == 2, 1, 0),
         Actif = ifelse(Status_Work %in% c(1,2), 1, 0))

# Calcul du chômage par diplôme
chomage_diplome <- pop_15plus %>%
  filter(!is.na(Educa_Level) & Educa_Level != "") %>%  # exclure valeurs manquantes
  group_by(Educa_Level) %>%
  summarise(
    Population_Active = sum(Actif, na.rm = TRUE),
    Chomeurs = sum(Chomeur, na.rm = TRUE),
    Taux_Chomage = round(100 * Chomeurs / Population_Active, 2)
  )

# Graphique
ggplot(chomage_diplome, aes(x=reorder(Educa_Level, -Taux_Chomage), y=Taux_Chomage)) +
  geom_bar(stat="identity", fill="steelblue") +
  labs(title="Taux de chômage par niveau d'éducation", x="Diplôme / Niveau d'éducation", y="Taux de chômage (%)") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle=45, hjust=1))


Chômage selon Secteur d’activité


In [ ]:
chomage_secteur <- df %>%
  filter(Status_Work %in% c(1,2)) %>%
  group_by(Sector_of_Activity) %>%
  summarise(
    Population_Active = n(),
    Chomeurs = sum(Status_Work == 2, na.rm=TRUE),
    Taux_Chomage = round(100 * Chomeurs / Population_Active,2)
  )

chomage_secteur


chômage par âge

In [ ]:
# Créer colonnes Actif et Chomeur
pop_15plus <- pop_15plus %>%
  mutate(
    Chomeur = ifelse(Status_Work == 2, 1, 0),   # 2 = chômeur
    Actif = ifelse(Status_Work %in% c(1,2), 1, 0)  # 1 ou 2 = actif
  )

# Créer des tranches d'âge
pop_15plus <- pop_15plus %>%
  mutate(Age_group = cut(Age, breaks = seq(15, 65, by = 5), right = FALSE))

# Taux de chômage par tranche d'âge
chomage_age <- pop_15plus %>%
  group_by(Age_group) %>%
  summarise(
    Population_Active = sum(Actif, na.rm = TRUE),
    Chomeurs = sum(Chomeur, na.rm = TRUE),
    Taux_Chomage = round(100 * Chomeurs / Population_Active, 2)
  )

# Graphique
ggplot(chomage_age, aes(x = Age_group, y = Taux_Chomage)) +
  geom_bar(stat = "identity", fill = "tomato") +
  labs(
    title = "Taux de chômage par tranche d'âge (15+)",
    x = "Tranche d'âge",
    y = "Taux de chômage (%)"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))
